In [3]:
import requests,time,json, pandas as pd
from datetime import datetime,timedelta,timezone
####Le Scripping de nabady

url = "https://api.nabady.ma/api/praticien_centre_soins/medecin/search"
headers = {
    "accept": "application/json, text/plain, */*",
    "accept-language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
    "cache-control": "no-cache",
    "content-type": "application/json",
    "origin": "https://nabady.ma",
    "pragma": "no-cache",
    "referer": "https://nabady.ma/",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/138.0.0.0 Safari/537.36",
}
page=1
total=0
limite=datetime.now(timezone.utc)-timedelta(days=45)
limite_=limite.replace(tzinfo=None)
ids_uniques=set()
all_Medecins=[]
date=datetime.now()
Duree=45
payload={
    "consultation":"15",
    "page":page,
    "result":200,
    "isIframe":False,
    "referrer":"",
    "domain":"ma"
}
while True:
    print(f"Traitement de la page : {page}")
    payload['page']=page  
    response=requests.post(url,headers=headers,json=payload,timeout=10)
    if response.status_code!=200:
        print("Erreur HTTP",response.status_code)
        break
    data_response=response.json().get("data",[])
    if not data_response:
        print("Fin des resultats")   
        break 
    for element in data_response:
            bloc=element.get("0",{})
            praticien=bloc.get("praticien",{})    
            if not praticien:
                continue
            Statut=""
            try:
                raw_connexion=praticien.get("lastcnx","")
                if raw_connexion is not None:
                    time_connexion=datetime.fromisoformat(raw_connexion)
                    if time_connexion.tzinfo is not None:
                        time_connexion=time_connexion.replace(tzinfo=None)
                        if time_connexion>=limite_:
                            Statut="Actif"
                        else:
                            Statut="Passif"  
                else:
                    Statut="Fictif"
                    time_connexion=datetime(1900,1,1)
            except Exception as e:
                Statut=""
            id_medecin=praticien.get("id","")
            civilite=praticien.get("civilite","")or ""
            prenom=praticien.get("firstname","")or ""
            nom=praticien.get("lastname","")or""
            email=praticien.get("email","")or ""
            if email:
                email=email.strip()
            sexe=praticien.get("sexe","")
            tel=praticien.get("tel","")
            Fix=praticien.get("fix","")
            adresse=praticien.get("adresse","")
            ville=praticien.get("ville",{})or{}
            ville_name=ville.get("name","")
            ville_id=ville.get("id","")
            try:
                date_naissance=praticien.get("dateNaissance","") 
                if date_naissance is not None:
                    time_date_naissance = datetime.fromisoformat(date_naissance)
                    if time_date_naissance.tzinfo is not None:
                        time_date_naissance = time_date_naissance.replace(tzinfo=None)
                else:
                    time_date_naissance=""
            except Exception as e:
                time_date_naissance=""
            raw_presentation=praticien.get("presentation","")
            presentation=raw_presentation.replace("\n"," ").strip() if isinstance(raw_presentation,str)else " "
            photo=praticien.get("img","")
            acces_specialite=praticien.get("centreSoinSpecialiteUsers",[])
            for item in acces_specialite:
                specialite=item.get("specialite",{})
                specialite_name=specialite.get("name","")
                specialite_id=specialite.get("id","")
            acces_langues=praticien.get("langue",[])
            langues=[c.get("langue")for c in acces_langues if c.get("langue")]
            if id_medecin not in ids_uniques:
                ids_uniques.add(id_medecin)
                all_Medecins.append({
                "ID Medecin":id_medecin,
                "Statut":Statut,
                "Civilite":civilite,
                "Nom":nom,
                "Prénom":prenom,
                "Sexe":sexe,
                "Date de Naissance":time_date_naissance,
                "Email":email,
                "Téléphone":tel,
                "Fix":Fix,
                "ID Spécialité":specialite_id,
                "Spécialité":specialite_name,
                "ID Ville":ville_id,
                "Ville":ville_name,
                "Photo URL":photo,
                "Adresse":adresse,
                "Présentation":presentation,
                "Langues Parlées":langues,
                "Date de Connexion":time_connexion,
                "Duree":Duree,
                "Date Extraction":date  
            })
                total+=1
                print( "1","|Prénom-",Statut,"|",Fix,"|" ,"|",limite_,time_connexion)
    print(f"Total dans la page{total}")
    time.sleep(2)
    page+=1  
    dfNabady=pd.DataFrame(all_Medecins)
    dfNabady.to_excel(r"C:\Users\HP\Desktop\Data_Folder\Fichier_Nabady_ma_Avant_Nettoyage.xlsx",index=False)
  


Traitement de la page : 1
1 |Prénom- Actif | 0522906363 | | 2025-06-23 15:47:24.665704 2025-07-31 15:33:49
1 |Prénom- Actif | 0530117870 | | 2025-06-23 15:47:24.665704 2025-08-07 04:52:55
1 |Prénom- Actif | 0642955995 | | 2025-06-23 15:47:24.665704 2025-08-07 16:47:08
1 |Prénom- Actif | 0624202808 | | 2025-06-23 15:47:24.665704 2025-08-07 16:25:42
1 |Prénom- Actif | 0522503023 | | 2025-06-23 15:47:24.665704 2025-08-07 09:32:21
1 |Prénom- Actif | 0530583099 | | 2025-06-23 15:47:24.665704 2025-08-07 16:36:46
1 |Prénom- Actif | 0520543420 | | 2025-06-23 15:47:24.665704 2025-08-07 16:27:28
1 |Prénom- Actif | 0539348688 | | 2025-06-23 15:47:24.665704 2025-08-07 16:32:53
1 |Prénom- Actif | 0661255996 | | 2025-06-23 15:47:24.665704 2025-08-07 16:27:02
1 |Prénom- Actif | None | | 2025-06-23 15:47:24.665704 2025-08-07 12:35:30
1 |Prénom- Actif | 0522863369 | | 2025-06-23 15:47:24.665704 2025-08-07 14:16:22
1 |Prénom- Actif | 0522900055 | | 2025-06-23 15:47:24.665704 2025-08-07 10:24:42
1 |Préno

In [27]:
dfNabady

,ID Medecin,Statut,Civilite,Nom,Prénom,Sexe,Date de Naissance,Email,Téléphone,Fix,...,Spécialité,ID Ville,Ville,Photo URL,Adresse,Présentation,Langues Parlées,Date de Connexion,Duree,Date Extraction
0,5224,Actif,Dr,AOUBEID,Amina,Femme,1979-12-31 00:00:00,aoubeidamina8@gmail.com,0657877488,0522906363,...,pédiatrie,168,Casablanca,None,"bd Oued Moulouya , lotiss. Selouane, imm. Offi...",Dr. AOUBEID Amina est pédiatre à Casablanca. M...,"[Arabe, Français]",2025-07-31 15:33:49,45,2025-08-04 11:04:35.082983
1,4532,Actif,Dr,EL HADI,Hind,Femme,1981-02-03 00:00:00,dr.elhadihind@gmail.com,0600666131,0530117870,...,dentaire,191,Kénitra,user/2023-04-1015-00-14963939125,"Avenue Mohamed V, 465BIS, Résidence Nezha, 2èm...",Dr. EL HADI Hind est dentiste à Kénitra. Membr...,"[Arabe, Français, Anglais, Espagnol]",2025-08-03 21:45:32,45,2025-08-04 11:04:35.082983
2,26238,Actif,Dr,BENYACINE,Haytem,Homme,1990-03-29 00:00:00,haytem.benyacine@yahoo.fr,0642955995,0642955995,...,oto-rhino-laryngologie(ORL),274,TANGER,None,App n°10 3éme étage avenue Pasteur n°35 Tanger,Oto-Rhino-Laryngologie et chirurgie cervico-fa...,"[Arabe, Français, Anglais]",2025-08-04 09:57:35,45,2025-08-04 11:04:35.082983
3,16832,Actif,Dr,MARGHADI,Amal,Femme,1995-05-02 00:00:00,amal.marghadi@gmail.com,0700151995,0624202808,...,médecine générale,195,Marrakech,None,ourika douar lhajeb,Generaliste,"[Arabe, Amazigh, Français]",2025-08-04 10:02:07,45,2025-08-04 11:04:35.082983
4,73,Actif,Dr,BAGHDADI,Najib,Homme,1980-12-31 00:00:00,baghdadi.56@hotmail.fr,0661801648,0522503023,...,oto-rhino-laryngologie(ORL),168,Casablanca,user/2023-03-0314-16-11370971316,Angle Bd Al Qods Et Bd Haifa Rés Riad El Qods ...,Dr. BAGHDADI Najib est oto-rhino-laryngologue ...,"[Arabe, Français]",2025-08-02 12:17:11,45,2025-08-04 11:04:35.082983
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,1604689,Actif,Dr,AABAD,Salah Eddine,homme,,dr.aabadsalah@gmail.com,0600235953,0539896060,...,médecine générale,2543,Bab Taza,None,"1er étage, quartier chair el hamrae (Près de p...",Dr. AABAD Salah eddine est médecin généralist...,"[Arabe, Français]",2025-06-25 15:01:58,45,2025-08-04 11:04:35.082983
2266,1619607,Actif,Dr,ZOUINI,Sara,Femme,,zouinisara2017@gmail.com,0661071152,0661071152,...,pédiatrie,2483,Skhirat,None,"227, Lotissement Ain Al Hayat 1, 2ème étage",Dr. MELOUK Soufiane est pédiatre à Skhirat. M...,"[Arabe, Français]",2025-07-22 12:49:37,45,2025-08-04 11:04:35.082983
2267,1817635,Actif,Dr,EZZAKI,Hassan,Homme,,hassanezzaki3@gmail.com,0661062880,0522710539,...,médecine générale,168,Casablanca,None,"265, rue lalla asmae, groupe 6, sidi moumen jdid",,"[Arabe, Français]",2025-07-22 14:48:13,45,2025-08-04 11:04:35.082983
2268,1824795,Actif,Dr,Hwiri,Imad Eddine,Homme,,imaddoc@outlook.com,0656608941,0656608941,...,pédiatrie,195,Marrakech,None,Les Portes de Marrakech tranche 13 imm 3 App N°1,,"[Arabe, Français]",2025-07-29 14:28:55,45,2025-08-04 11:04:35.082983


In [25]:

import re,pandas as pd
df1=dfNabady
#------------------------Nettoyer Civilite-------------------------------------
df1["Civilite"]=df1["Civilite"].astype(str).str.replace(".","")
df1=df1.rename(columns={"Civilite":"Civilité"})
#------------------Nettoyer Nom et Prenom---------------------
df1["Nom"]=df1["Nom"].str.title().str.strip()
df1["Prénom"]=df1["Prénom"].str.title().str.strip()
#----------------------Nettoyage du Sexe----------------------
df1["Sexe"]=df1["Sexe"].astype(str).str.strip().str.capitalize().replace("none","").replace("NAN","").replace("nan","").replace("Nan","").replace("None","")
#-------Nettoyage de Spécialité------------------------
df1=df1.rename(columns={"Nom Ville":"Ville","Specialité":"Spécialité"})
classification = {
    "Anesthésiologie": [
        "réanimateur pédiatrique","réanimateur médical","anesthésie", "anesthésie-réanimation","anesthésiste-réanimateur"
    ],
    "Andrologie": [
        "andrologie","andrologue"
    ],
    "Cardiologie": [
        "cardiologie", "cardiologue", "cardiologie interventionnelle", "rythmologie interventionnelle","rythmologue interventionnel"
    ],
    "Chirurgie": [
        "chirurgie" ,"proctologie","proctologue"
    ],
    "Chirurgie cardiaque": [
        "chirurgie cardio", "chirurgie cardio-vasculaire"
    ],
    "Chirugie esthétique, plastique et reconstructive": [
        "chirurgie esthétique", "chirurgie plastique", "chirurgie réparatrice","médecin esthétique",
        "chirurgie plastique, reconstructrice et esthétique","médecine esthétique","médecine au laser"
    ],
    "Chirurgie générale": [
        "chirurgie générale", "chirurgie general", "chirurgie digestive", "chirurgie cancérologique","Proctologie"
    ],
    "Chirurgie maxillo-faciale": [
        "chirugien cervico-facial","stomatologue","chirurgie maxillo", "chirurgie cervico-faciale", "oto-rhino-laryngologie et chirurgie cervico-faciale","Chirugie Cervico-Faciale","chirurgien cervico-facial"
    ],
    "Chirurgie pédiatrique": [
        "chirurgie pédiatrique"
    ],
    "Chirurgie thoracique": [
        # à compléter si des mots-clés spécifiques apparaissent dans tes données
    ],
    "Chirurgie vasculaire": [
        "phlébologue","chirurgie vasculaire","Phlébologie","angiologue","audioprothésiste"
    ],
    "Neurochirurgie": [
        "neurochirurgie"
    ],
    "Dermatologie": [
        "dermatologue","dermatologie", "dermatologue","dermatologue pédiatrique","dermatologie et venereologie",
        "dermatologie pédiatrique"
    ],
    "Nutrition":["diététique","nutritionniste","nutrition"],
    "Endocrinologie": [
        "diabétologue","diététicien","endocrinologie", "endocrinologue", "endocrinologie et métabolismes",
        "diabétologie nutritionnelle"
    ],
    "Gastro-entérologie": [
        "gastro-entérologue","gastro-entérologue","gastroentérologie", "gastrologie", "gastrologue",
        "hépatogastroentérologie","du pancréas et des voies biliaires", "hepatogastroenterologue","Hépato-Gastro-Entérologie"
    ],
    "Gériatrie": [
        "gérontologue - gériatre","gériatrie", "gerontologie", "la gériatrie","gérontologie"
    ],
    "Gynécologie": [
        "gynécologie", "gynécologue", "gynécologie obstétrique", "gynecologie-obstetrique","gynecologue","perineologue"
    ],
    "Hématologie": [
        "hématologie","hématologue"
    ],
    "Hépatologie":[
        "hépatologie","hépatologue"  # si présent explicitement
    ],
    "Infectiologie": [
        "infectiologie","maladies infectieuses" # à adapter selon tes données
    ],
    "Médecine aiguë": [
        "médecine d'urgence","médecin urgentiste",
    ],
    "Médecine du travail": [
        "médecine du travail","médecin du travail"
    ],
    "Biologie médicale":["laboratoire d'analyses de biologie médicale","biologiste medicale","biologiste (laboratoire d'analyse)","biologie médicale"],
    "Médecine légale":[
        "médecin légiste",
    ],
    "Médecine générale": [
        "médecin généraliste","médecine générale", "omnipraticien", "médecine de famille","généraliste","médecin de famille"
    ],
    "Médecine interne": [
        "médecine interne","médecin interniste","médecin du sommeil","angiologie"
    ],
    "Médecine nucléaire": [
        "médecine nucléaire","médecin nucléaire"
    ],
    "Médecine palliative": [
        "médecine palliative","algologue - traitement de la douleur"
    ],
    "Médecine physique": [
        "ergothérapie","orthophoniste","posturologue","podologue","chiropracteur","Kinesitherapeute","médecine physique", 
        "médecine physique et de réadaptation", "mpr","kinésithérapie","orthophonie","Podologie","kinésithérapeute",
        "kinésithérapeute du sport","médecin physique et de réadaptation","ergothérapeute","médecin physique réadaptateur"
        
    ],
    "Médecine préventive": [
        "médecin du sport","ostéopathe","homéopathe","médecine préventive", "santé publique", "médecine du sport", "médecine aéronautique et spatiale","Naturopathie",
        "Allergologie","Naturopathie","Médecine Chinoise(Mtc)","ostéopathie","homéopathie","acupuncture","allergologue","allergologue-pédiatrique",
        "tabacologue","naturopathe","sophrologue"
        
    ],
    "Néonatologie": [
        "néonatologie"
    ],
    "Néphrologie": [
        "néphrologie","centre de dialyse","néphrologue","néphrologue pédiatrique","neurophysiologiste"
    ],
    "Neurologie": [
        "neurologie","neurologue"
    ],
    "Odontologie": [
        "endodontiste","occlusodontiste","orthodontiste","parodontologiste","pédodontiste",
        "dentiste", "chirurgie dentaire", "chirurgie dentiste", "dentaire","implantologiste",
        "orthodontie", "parodontologie", "pédodontie"
    ],
    "Oncologie": [
        "oncologie", "cancérologie","oncologue - cancérologue","oncologue médical","cancérologue","oncologue"
    ],
    "Obstétrique": [
        "obstétrique", "sage femme","sage-femme"
    ],
    "Ophtalmologie": [
        "ophtalmologie", "orthoptie","ophtalmologue","ophtalmologue pédiatrique","orthoptiste","néonatologiste"
    ],
    "Orthopédie": [
        "traumato-orthopédiste pédiatrique","traumatologue-orthopédiste","Orthopédie traumatolo","orthopédie", "traumatologie", "traumato - orthopedie",
        "chirurgie orthopédiste", "chirurgie traumatologie-orthopédie", "chirurgie orthopédique","Perineologue","Orthopédiste Traumatologue"
    ],
    "Oto-rhino-laryngologie": [
        "orl","oto-rhino-laryngologie","oto-rhino-laryngologie(orl)","oto-rhino-laryngologie et chirurgie cervico-faciale"
    ],
    "Pédiatrie ": [
        "pédiatrie","pédiatre"
    ],
    "Anatomopathologie": [
    "anatomopathologiste", "anatomopathologie", "médecin anatomopathologiste"
    ],
    "Aneumologie ": [
        "pneumologie","pneumologue"
    ],
    "Psychiatrie ": [
        "psychiatrie","psychanalyste","coach développement personnel","hypnothérapeute","pédopsychiatre","psychiatre","psychologue","psychologue","psychomotricien","psychothérapeute","sexologue","sophrologue"
        "psychiatrie","sexologie","neuropsychiatrie", "psychologie", "psychomotricité", "psychothérapie", "pedopsychiatrie","Addictologie","psycologue","addictologue"
    ],
    "Radiologie ": [
        "radiologie","radiologue"
    ],
    "Radiothérapie ": [
        "radiothérapeute","radiothérapie", "oncologie radiothérapie", "oncologie option radiothérapie"
    ],
    "Rhumatologie": [
        "rhumatologie","rhumatologue"
    ],
    "Urologie": [
        "urologie", "chirurgie urologique","urologue","urologue pédiatrique"
    ]}
def reclasser_specialite(specialite):
    specialite = str(specialite).strip().lower().replace("nan","")
    for categorie, mots_cles in classification.items():
        for mot in mots_cles:
            if mot.lower() in specialite:
                return categorie
    return specialite if specialite else ""

df1["Spécialité"]=df1["Spécialité"].apply(reclasser_specialite)
df1["Spécialité"]=df1["Spécialité"].astype(str).str.title()

#----------------------Nettoyage de la ville------------------
df1["Ville"] = (df1["Ville"].astype(str).str.title()
                .str.replace("Nan","").str.replace("None","")
                .str.replace("Fquih Ben Salah","Fkih Ben Salah")
                .str.replace("Khemisset","Khémisset")
                .str.replace("Taroudannt","Taroudant")
                .str.replace("Mohammadia","Mohammédia")
                .str.replace("Skhirat","Skhirate")
                .str.replace("Skhiratee","Skhirate")
)
# ------------------------------------------Nettoyage de Numero de telephone Marocain------------------------
def Nettoyage_phone(tel):
    if pd.isna(tel):
        return ""
    tel=str(tel).replace("-","").replace(" ","")
    numeros=re.findall(r'\d{8,15}',tel)
    resultats=[]
    for n in numeros:
        if n.startswith("212"):
            n="+" +n
        elif n.startswith("0"):
            n="+212"+n[1:]
        elif n.startswith(("6","7","5")):
            n="+212"+n
        if n not in ["none","NAN","+212","","Nan"]:
            resultats.append(n)
    return ",".join(resultats)
df1["Téléphone"] = df1["Téléphone"].apply(Nettoyage_phone)
df1["Fix"] = df1["Fix"].apply(Nettoyage_phone)
#------------------------Nettoyer la date de connexion et extraction---------------
df1["Date Extraction"]=df1["Date Extraction"].dt.strftime("%Y-%m-%d %H:%M:%S")
col_date=["Date Extraction","Date de Connexion"]
for colD in col_date:
    df1[colD]=pd.to_datetime(df1[colD],format="%Y-%m-%d %H:%M:%S")
#-----------------------Nettoyage des Langues Parlées-----------------
df1["Langues Parlées"]=(df1["Langues Parlées"]
                        .astype(str).str.title()
                        .str.replace("عربي","Arabe")
                        .str.replace("'","")
                        .replace("Nan","")
                        .replace("nan","")
                        .replace("NAN","")
                        .replace("none","")
                        .str.replace("[","")
                        .str.replace("]",""))

#------------------------Les dérnières modifications------------------------------------
columns=["ID Medecin","Statut","Civilité","Email","Photo URL"]
for col in columns:
    df1[col]=df1[col].fillna("")
    df1[col]=df1[col].astype(str).str.replace(r'[\u0600-\u06FF\u0750-\u077F\u08A0-\u08FF]+','', regex=True).str.replace("none","").str.replace("Nan","").str.replace("nan","").str.replace(r"[-<>]","",regex=True)

order_columns=["ID Medecin","Statut","Civilité","Prénom","Nom","Sexe","Spécialité","Ville","Téléphone","Fix","Email","Date de Naissance","Langues Parlées","Photo URL","Duree","Date de Connexion","Date Extraction"]
df1=df1[order_columns]
df1.to_excel(r"C:\Users\HP\Desktop\Data_Folder\Fichier_Nabady_ma_Apres_Nettoyage.xlsx", index=False)
df1

,ID Medecin,Statut,Civilité,Prénom,Nom,Sexe,Spécialité,Ville,Téléphone,Fix,Email,Date de Naissance,Langues Parlées,Photo URL,Duree,Date de Connexion,Date Extraction
0,5224,Actif,Dr,Amina,Aoubeid,Femme,Pédiatrie,Casablanca,+212657877488,+212522906363,aoubeidamina8@gmail.com,1979-12-31 00:00:00,"Arabe, Français",,45,2025-07-31 15:33:49,2025-08-04 11:04:35
1,4532,Actif,Dr,Hind,El Hadi,Femme,Odontologie,Kénitra,+212600666131,+212530117870,dr.elhadihind@gmail.com,1981-02-03 00:00:00,"Arabe, Français, Anglais, Espagnol",user/20230410150014963939125,45,2025-08-03 21:45:32,2025-08-04 11:04:35
2,26238,Actif,Dr,Haytem,Benyacine,Homme,Oto-Rhino-Laryngologie,Tanger,+212642955995,+212642955995,haytem.benyacine@yahoo.fr,1990-03-29 00:00:00,"Arabe, Français, Anglais",,45,2025-08-04 09:57:35,2025-08-04 11:04:35
3,16832,Actif,Dr,Amal,Marghadi,Femme,Médecine Générale,Marrakech,+212700151995,+212624202808,amal.marghadi@gmail.com,1995-05-02 00:00:00,"Arabe, Amazigh, Français",,45,2025-08-04 10:02:07,2025-08-04 11:04:35
4,73,Actif,Dr,Najib,Baghdadi,Homme,Oto-Rhino-Laryngologie,Casablanca,+212661801648,+212522503023,baghdadi.56@hotmail.fr,1980-12-31 00:00:00,"Arabe, Français",user/20230303141611370971316,45,2025-08-02 12:17:11,2025-08-04 11:04:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2265,1604689,Actif,Dr,Salah Eddine,Aabad,Homme,Médecine Générale,Bab Taza,+212600235953,+212539896060,dr.aabadsalah@gmail.com,,"Arabe, Français",,45,2025-06-25 15:01:58,2025-08-04 11:04:35
2266,1619607,Actif,Dr,Sara,Zouini,Femme,Pédiatrie,Skhirate,+212661071152,+212661071152,zouinisara2017@gmail.com,,"Arabe, Français",,45,2025-07-22 12:49:37,2025-08-04 11:04:35
2267,1817635,Actif,Dr,Hassan,Ezzaki,Homme,Médecine Générale,Casablanca,+212661062880,+212522710539,hassanezzaki3@gmail.com,,"Arabe, Français",,45,2025-07-22 14:48:13,2025-08-04 11:04:35
2268,1824795,Actif,Dr,Imad Eddine,Hwiri,Homme,Pédiatrie,Marrakech,+212656608941,+212656608941,imaddoc@outlook.com,,"Arabe, Français",,45,2025-07-29 14:28:55,2025-08-04 11:04:35


In [26]:
#Insertion dans MongoDB
from ConnexionMongo_DB import get_db
from insertion import inserer_dataframe
db=get_db()
inserer_dataframe(df1,db,"nabadyVersion_04_08")

succes
2270 documents insérés dans 'nabadyVersion_04_08'


In [ ]:
def scraper_et_inserer():
    # 🛑 Simule deux médecins
    medecins = [ ... ]
    
    # Ajoute la date d’extraction à chaque document
    now = datetime.now()
    for m in medecins:
        m["date_extraction"] = now
    
    # Connexion MongoDB et insertion
    db["medecins_historique"].insert_many(medecins)


✅ 0 éléments ajoutés.


In [12]:
#Identifier les elements supprimés avant la date courante
date_courante="2025-07-23 04:19:07"
df_ancien=df1[df1["Date Extraction"]<date_courante]
#Récuperer les IDs
ids_avantS=set(df_ancien["ID Medecin"])
ids_courantS=set(df1[df1["Date Extraction"]==date_courante]["ID Medecin"])
ids_supprimes=ids_avantS-ids_courantS
df_supprimes=df_ancien[df_ancien["ID Medecin"].isin(ids_supprimes)]
print(f"le nombre des elements supprimés est {len(df_supprimes)}")
df_supprimes


le nombre des elements supprimés est 0


,ID Medecin,Statut,Civilité,Nom Complet,Sexe,Spécialité,Ville,Téléphone,Fix,Email,...,Langues Parlées,Photo URL,Adresse,Diplomes,Formations,Certificats,Soins et Actes,Duree,Date de Connexion,Date Extraction


In [90]:
a_str="2021-02-01"
a_time=pd.to_datetime(a_str,format="%Y-%m-%d")
print(a_time)

2021-02-01 00:00:00
